This script will assess daily ensemble hindcast data provided by the NWS. It will read the daily .xml files, calculate the cumulative volume over the first 5 days, for each of the 60 forcasts published  each day, and report those volumes to a .csv.

In [2]:
#User Entry
targetFile = r"C:\Users\q0hecbbb\Projects\Hindcast Data\WorkingFolder\1985020812_LAMC1_HEFS.xml"


In [3]:
#Imported Libraries
import pandas as pd
import xml.etree.ElementTree as et
import io
import statistics
import numpy as np

# Methods
def kcfs2acFtHour(foo):
    return (foo*1000*3600/43560)

# Read XML
tree = et.parse(targetFile)
root = tree.getroot()
seriesCount = 1
data = []
for eachSeries in root[1:]:
    seriesID = seriesCount
    seriesCount = seriesCount + 1
    for eachEvent in eachSeries[1:]:
        eachEvent.attrib['seriesID'] = seriesID
        data.append(eachEvent.attrib)
df = pd.DataFrame(data)

#Save forecast Date
forecastDate = df.date[0]

#Prep Dataframe
df['value'] = df['value'].astype(float)
df['value'] = df['value'].apply(kcfs2acFtHour)
df = df.set_index('seriesID')

#Main
numSeries = (seriesCount)
day1List = []
day2List = []
day3List = []
day4List = []
day5List = []
outputList = []
masterDictionary = {}


for num in range(1, 62):
    hours = 24
    
    currentSeries = df.loc[num]
    currentSeries = currentSeries.iloc[1:]
    
    day1List.append(currentSeries.value.head(hours).sum(0))
    hours = hours + 24
    day2List.append(currentSeries.value.head(hours).sum(0))
    hours = hours + 24
    day3List.append(currentSeries.value.head(hours).sum(0))
    hours = hours + 24
    day4List.append(currentSeries.value.head(hours).sum(0))
    hours = hours + 24
    day5List.append(currentSeries.value.head(hours).sum(0))
    
listOflists = [day1List, day2List, day3List, day4List, day5List]
for eachList in listOflists:
        outputList.append(np.percentile(eachList, 50))
    
masterDictionary = {forecastDate : outputList}

In [4]:
dfout = pd.DataFrame(masterDictionary)
dfout

,1985-02-08
0,2248.616665
1,2683.014568
2,3009.659339
3,3969.729011
4,4596.348626
